In [1]:
"""
References: 
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html
"""

import cv2
import math
import numpy as np
from affinity_util import *

"Values taken from original author's implementation"
NORMALIZE_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32) * 255.0
NORMALIZE_VARIANCE = np.array([0.229, 0.224, 0.225], dtype=np.float32) * 255.0

def watershed(inp):
    gray = inp.copy()
    img = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    _ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _ret, sure_fg = cv2.threshold(dist_transform, 0.2 * dist_transform.max(), 255, cv2.THRESH_BINARY)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    markers = cv2.watershed(img, markers)
    
    return markers

def find_char_box(markers):
    boxes = []
    marker_count = np.max(markers)
    for i in range(2,marker_count+1):
        cnt = np.swapaxes(np.array(np.where(markers == i)), axis1=0, axis2=1)[:, ::-1]
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        boxes.append(box)
    return boxes

def dist(p1, p2):
    return math.sqrt(math.pow((p2[0] - p1[0]), 2) + math.pow((p2[1] - p1[1]), 2))

def crop_image(src, points, dst_height=None):
    """
    Crop image with box points.
    src - Input image
    points - coordinates of box
    dst_height - optional parameter for fixed height of box 
    (note that this is not same as variable h but actual height of character)
    """
    src_image = src.copy()
    src_points = np.float32(points)
    w = round((dist(points[0], points[1]) + dist(points[2], points[3])) / 2)
    h = round((dist(points[1], points[2]) + dist(points[3], points[0])) / 2)
    
    #set to fixed height
    if dst_height is not None:
        ratio = dst_height / min(w, h)
        w = int(w * ratio)
        h = int(h * ratio)
        
    #get cropped image
    crop_points = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    perspective_mat = cv2.getPerspectiveTransform(src=src_points, dst=crop_points)
    dst = cv2.warpPerspective(src_image, perspective_mat, (w, h), borderValue=0, borderMode=cv2.BORDER_CONSTANT)
    return dst, src_points, crop_points

def un_warping(box, src_points, crop_points):
    """
    Unwarp the character bounding boxes.
    box - The character bounding box.
    src_points - Points before crop.
    crop_points - Points after crop.
    """
    perspective_mat = cv2.getPerspectiveTransform(src=crop_points, dst=src_points)
    new_box = list()
    for x, y in box:
        new_x = int(np.dot(perspective_mat[0], [x,y,1]) /np.dot(perspective_mat[2], [x,y,1]))
        new_y = int(np.dot(perspective_mat[1], [x,y,1]) /np.dot(perspective_mat[2], [x,y,1]))
        new_box.append([new_x, new_y])
    return new_box

def divide_region(box, length):
    """
    If confidence < 0.5, to obtain character bounding boxes
    """
    if length == 1:
        return [box]
    
    char_boxes = []
    p1, p2, p3, p4 = box
    if dist(p1, p2) + dist(p3, p4) > dist(p2, p3) + dist(p4, p1):
        x_start1 = p1[0]
        y_start1 = p1[1]
        x_start2 = p4[0]
        y_start2 = p4[1]
        x_offset1 = (p2[0] - p1[0]) / length
        y_offset1 = (p2[1] - p1[1]) / length
        x_offset2 = (p3[0] - p4[0]) / length
        y_offset2 = (p3[1] - p4[1]) / length
    else:
        x_offset1 = (p4[0] - p1[0]) / length
        y_offset1 = (p4[1] - p1[1]) / length
        x_offset2 = (p3[0] - p2[0]) / length
        y_offset2 = (p3[1] - p2[1]) / length
        x_start1 = p1[0]
        y_start1 = p1[1]
        x_start2 = p2[0]
        y_start2 = p2[1]
        
    for i in range(length):
        char_boxes.append([
            [round(x_start1 + x_offset1 * i), round(y_start1 + y_offset1 * i)],
            [round(x_start1 + x_offset1 * (i + 1)), round(y_start1 + y_offset1 * (i + 1))],
            [round(x_start2 + x_offset2 * i), round(y_start2 + y_offset2 * i)],
            [round(x_start2 + x_offset2 * (i + 1)), round(y_start2 + y_offset2 * (i + 1))]
        ])

    return char_boxes

def enlarge_char_box(char_box, ratio):
    x_center, y_center = np.average(char_box[:, 0]), np.average(char_box[:, 1])
    char_box = char_box - [x_center, y_center]
    char_box = char_box * ratio
    char_box = char_box + [x_center, y_center]
    return char_box


def enlarge_char_boxes(char_boxes, crop_box):
    char_boxes = np.reshape(np.array(char_boxes), newshape=(-1, 4, 2))
    left, right, top, bottom = np.min(char_boxes[:, :, 0]), np.max(char_boxes[:, :, 0]), \
                               np.min(char_boxes[:, :, 1]), np.max(char_boxes[:, :, 1])
    width, height = crop_box[2, 0], crop_box[2, 1]
    offset = np.min([left, top, width - right, height - bottom])
    ratio = 1 + offset * 2 / min(width, height)
    char_boxes = np.array([enlarge_char_box(char_box, ratio) for char_box in char_boxes])
    char_boxes[:, :, 0] = np.clip(char_boxes[:, :, 0], 0, width)
    char_boxes[:, :, 1] = np.clip(char_boxes[:, :, 1], 0, height)
    return char_boxes

def conf(boxes, word):
    box_count = len(boxes)
    word_length = len(word)
    confidence = (word_length - min(word_length, abs(word_length - box_count))) / word_length
    return confidence

def gen(src,word_boxes,words):
    """
    src - Heat map generated from prediction
    word_boxes - all box annotations for the src image
    words - all texts in the src image
    
    Return:
    char_boxes_list: boxes for characters in the original image space (not src image)
    confidence_score: confidence mask for the given image - same size as src
    """
    char_boxes_list = []
    confidence_list = []
    for word_box,word in zip(word_boxes, words):
        img, src_points, crop_points = crop_image(src,word_box//2, dst_height=64.)
        markers = watershed(img)
        char_boxes = find_box(markers)
        confidence = conf(char_boxes, word)
        if confidence <= 0.5:
            confidence = 0.5
            char_boxes = divide_region(word_box, word_length)
            char_boxes = [reorder_points(region_box) for region_box in region_boxes]
        else:
            char_boxes = np.array(region_boxes) * 2
            char_boxes = enlarge_char_boxes(region_boxes, 2*crop_points)
            char_boxes = [un_warping(region_box, 2*src_points, 2*crop_points) for region_box in region_boxes]
            
        char_boxes_list.append(char_boxes)
        confidence_list.append(confidence)
        
    confidence_score = np.ones(src.shape[:2], dtype=np.float32)
    for word_box, confidence_value in zip(word_boxes, confidence_list):
        if confidence_value == 1:
            continue
        tmp_confidence_score = np.zeros(heat_map_size, dtype=np.uint8)
        cv2.fillPoly(tmp_confidence_score, [np.array(word_box)], 1)
        tmp_confidence_score = np.float32(tmp_confidence_score) * confidence_value
        confidence_score = \
            np.where(tmp_confidence_score > confidence_score, tmp_confidence_score, confidence_score)
        
    return char_boxes_list, np.array(confidence_score)